Script to extract ferret timeseries data into two columns of numbers.
Handles files with and without dates for x-values

In [1]:
import pandas as pd

# For bokeh plots
import bokeh #0.12.3
from bokeh.plotting import figure
from datetime import datetime as dt
from bokeh.models import DatetimeTickFormatter
from math import pi


In [3]:
# Read csv file output by ferret

df_orig = pd.read_csv('levitus.csv', delimiter=':', index_col=0)
df_orig.head(10)

,TEMPERATURE (DEG C)
VARIABLE,
FILENAME,levitus_climatology.cdf
FILEPATH,/homel/cnangini/.conda/envs/FERRET/share/fer_...
SUBSET,20 points (DEPTH (m))
LONGITUDE,110.4W(-110.4) to 30.2E(30.2) (XY ave)
LATITUDE,70.3S to 70.3N (XY ave)
0 / 1,17.32
10 / 2,17.25
20 / 3,17.07
30 / 4,16.76


In [31]:
# Read csv file output by ferret, skipping header
df = pd.read_csv('levitus.csv', delimiter=':', skiprows=5)
df.head(5)

,LATITUDE,70.3S to 70.3N (XY ave)
0,0 / 1,17.32
1,10 / 2,17.25
2,20 / 3,17.07
3,30 / 4,16.76
4,50 / 5,15.84


In [32]:
#Extract date string from col 0
s = df.ix[:, 0]
s.head(5)

0     0       /  1
1     10      /  2
2     20      /  3
3     30      /  4
4     50      /  5
Name:              LATITUDE , dtype: object

In [33]:
x = s.str.split(' /').str.get(0).str.split(' ').str.get(1)
x.head(5)

0     0
1    10
2    20
3    30
4    50
dtype: object

In [34]:
# Determine if x-col is a list of dates
months=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
dateFlag=0
for idx in range(12):
    if x[0].find(months[idx]) != -1:
        print("found month")
        dateFlag=1
print dateFlag 

0


In [35]:
if dateFlag==1:
    x = pd.to_datetime(x,infer_datetime_format=True)
x.head(5)

0     0
1    10
2    20
3    30
4    50
dtype: object

In [36]:
# Put x and y values in dataframe
dfer = pd.DataFrame()
dfer['xval'] = x
dfer['yval'] = df.ix[:, 1]
dfer.head(5)

,xval,yval
0,0,17.32
1,10,17.25
2,20,17.07
3,30,16.76
4,50,15.84


In [37]:
p = figure(title='A Bokeh plot',
        plot_width=700,plot_height=400)

In [38]:
p.line(dfer['xval'], dfer['yval'])

In [39]:
if dateFlag==1:
    p.xaxis.formatter=DatetimeTickFormatter(formats=dict(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    ))
p.xaxis.major_label_orientation = pi/4
p.yaxis.axis_label = "size"


In [40]:
# For plotting in notebook
from bokeh.io import output_file
from bokeh.charts import show
from bokeh.models import DatetimeTickFormatter

output_file('myplot2.html')
show(p)